In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import hues

In [19]:
# Define loss function
def loss_fun(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta) ** 2)) / len(X_b)
    except:
        hues.error('The return value is to large!!!')
        return np.inf


# Define the Find the derivative function
def get_der(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(X_b)

theta_history=[]
def gradient_descent(X_b, y, ini_theta, eta, n_iters=1e4, eps=1e-8):
    theta = ini_theta
    i = 0
    theta_history.append(ini_theta)

    while i < n_iters:
        gradient = get_der(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        theta_history.append(theta)

        if abs(loss_fun(theta, X_b, y) - loss_fun(last_theta, X_b, y)) < eps:
            break

        i += 1

    return theta

In [15]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [16]:
X, y = data, target
X = X[y < 50]
y = y[y < 50]
X.shape, y.shape

((490, 13), (490,))

In [17]:
X_b = np.hstack([np.ones([len(X), 1]), X])

In [18]:
# Split the Feature Dataset and Label Dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, = train_test_split(X_b, y, test_size=0.3, random_state=666)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((343, 14), (147, 14), (343,), (147,))

In [23]:
%%time
initial_theta = np.zeros(X_train.shape[1])
eta = 0.001

gradient_descent(X_train, y_train, initial_theta, eta)

CPU times: total: 93.8 ms
Wall time: 173 ms


C:\Users\myxc\AppData\Local\Temp\ipykernel_4432\2878627023.py:4: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta) ** 2)) / len(X_b)
C:\Users\myxc\AppData\Local\Temp\ipykernel_4432\2878627023.py:26: RuntimeWarning: invalid value encountered in double_scalars
  if abs(loss_fun(theta, X_b, y) - loss_fun(last_theta, X_b, y)) < eps:
C:\Users\myxc\AppData\Local\Temp\ipykernel_4432\2878627023.py:12: RuntimeWarning: overflow encountered in multiply
  return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(X_b)
C:\Users\myxc\AppData\Local\Temp\ipykernel_4432\2878627023.py:23: RuntimeWarning: invalid value encountered in subtract
  theta = theta - eta * gradient


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan])

In [24]:
calc_theta = gradient_descent(X_train, y_train, initial_theta, eta=1e-6)
y_predict = X_test.dot(calc_theta)

from sklearn.metrics import r2_score

r2_score(y_test, y_predict)

0.30594186189467965

In [25]:
%%time
calc_theta = gradient_descent(X_train, y_train, initial_theta, eta=1e-6, n_iters=1e6)
y_predict = X_test.dot(calc_theta)

from sklearn.metrics import r2_score

r2_score(y_test, y_predict)

CPU times: total: 6.28 s
Wall time: 17.2 s


0.7440227019523431

In [26]:
# Using data normalization
from sklearn.preprocessing import StandardScaler

scale_scaler = StandardScaler()
scale_scaler.fit(X_train)

StandardScaler()

In [41]:
X_train_std = scale_scaler.transform(X_train)
X_test_std = scale_scaler.transform(X_test)
X_train_std = np.hstack([np.ones([len(X_train), 1]), X_train_std])
X_test_std = np.hstack([np.ones([len(X_test), 1]), X_test_std])

In [42]:
%%time
initial_theta = np.zeros(X_train_std.shape[1])
calc_theta = gradient_descent(X_train_std, y_train, initial_theta, eta=0.001)
y_predict = X_test_std.dot(calc_theta)

from sklearn.metrics import r2_score

r2_score(y_test, y_predict)

CPU times: total: 78.1 ms
Wall time: 176 ms


0.7983573625036819

In [49]:
# Rewrite the function of gradient descent
def get_der(theta, X_b, y):
    rand_index = np.random.randint(len(X_b))
    X_b_i, y_i = X_b[rand_index], y[rand_index]
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2

theta_history = []
def gradient_descent(X_b, y, ini_theta, eta, n_iters=1e4, eps=1e-8):
    theta = ini_theta
    i = 0
    theta_history.append(ini_theta)
    t_0, t_1 = 5, 50

    def learning_rate(t):
        return t_0 / (t + t_1)

    while i < n_iters:
        gradient = get_der(theta, X_b, y)
        theta = theta - learning_rate(i) * gradient
        theta_history.append(theta)
        i += 1

    return theta

In [50]:
%%time
m = 100000

x = np.random.normal(size=m)
X = x.reshape(-1, 1)
X_b = np.hstack([np.ones([len(X), 1]), X])

y = 4 * x + 3 + np.random.normal(0, 3, size=m)

initial_theta = np.zeros(X_b.shape[1])
eta = 0.001

calc_theta = gradient_descent(X_b, y, initial_theta, eta)
calc_theta

CPU times: total: 31.2 ms
Wall time: 69.8 ms


array([3.01261573, 4.03667353])

In [51]:
theta_history

[array([0., 0.]),
 array([-0.20286549,  0.22300886]),
 array([1.72015117, 2.76390422]),
 array([2.29246922, 2.57419784]),
 array([2.17770594, 2.57007606]),
 array([2.51991887, 2.70560661]),
 array([1.92738612, 1.88905861]),
 array([3.39153196, 5.88829583]),
 array([3.47740565, 5.89506986]),
 array([3.93488011, 5.2826789 ]),
 array([3.97582198, 5.2290793 ]),
 array([4.19960978, 4.74351947]),
 array([3.28811975, 4.5399002 ]),
 array([2.51458868, 4.01224852]),
 array([2.23057418, 4.02025009]),
 array([2.17561982, 4.06132124]),
 array([2.2666842 , 3.96617263]),
 array([2.386421  , 4.05006514]),
 array([2.93834012, 3.72871806]),
 array([2.86389729, 3.74869859]),
 array([2.62468368, 3.66463385]),
 array([2.59708987, 3.6839583 ]),
 array([3.25549116, 3.88640837]),
 array([3.23576105, 3.87279481]),
 array([3.55098406, 4.14738408]),
 array([2.74343369, 4.03231142]),
 array([3.29429549, 3.91840371]),
 array([3.35301068, 3.89423196]),
 array([3.20816624, 3.89568804]),
 array([3.12420306, 4.001692